# Couverture des bs
Partie de Clément

In [1]:
import numpy as np
import heapq
import random

# Trajets optimaux entre les bs et les fermes

Etant donné des positions des bs et des fermes sur la carte, on cherche le plus court chemin entre les fermes et les bs et entre les bs.

Ce n'est pas encore sûr que ça arrivera ou que ça sera utile et donc implémenté mais, s'il le faut, toutes les fermes en dehors de la carte démographique seront ramenés à des points de la carte auquel on ajoutera un poids initial de trajet.

Ensuite, étant donné la densité de routes, et surtout pour raccourcir énormément les calculs à chaque fois et rendre bien plus simple l'implémentation et la lisibilité, nous allons juste calculer les distances à vol d'oiseau.

In [2]:
# Créer une file de priorité vide
priority_queue = []

# Ajouter des éléments avec des priorités
heapq.heappush(priority_queue, (3, "Trois"))
heapq.heappush(priority_queue, (1, "Un"))
heapq.heappush(priority_queue, (2, "Deux"))

# Retirer l'élément avec la plus haute priorité (la plus petite valeur)

print(priority_queue)  # Affiche (1, "Un")


[(1, 'Un'), (3, 'Trois'), (2, 'Deux')]


In [3]:
dmax = 100

def distance(u,v):
    return np.sqrt((u[0]-v[0])**2 + (u[1]-v[1])**2)

def distances_orientees(fermes, bs, n_links): 
    """ Permet de créer une liste d'adjacence avec au plus n_links liens entre les éléments, en triant selon les distances.
    Le tableau rendu est de la forme [[f1, [[bi,d1i],[bj,d1j],...]], [f2, [[bi,d2i],[bj,d2j],...]]], ...] """
    fermes_bio = []
    bio_bio = []

    # Pour les fermes
    for f in fermes:
        tab = []
        for b in bs:
            d = distance(f,b)
            p = dmax - d
            if tab == []:
                heapq.heappush(tab, (p, [b,d]))
            elif tab[1][1] > d:
                if len(tab) == n_links :
                    heapq.heappop(tab)
                heapq.heappush(tab, (p, [b,d]))
        fermes_bio.append([f, tab])

    # Pour les bs
    for f in bs: 
        # On fait le choix de ne pas considérer de symétrie entre les bs : un b peut être le plus proche
        # d'un autre de son point de vue, mais très éloigné de l'autre par rapport à d'autres
        tab = []
        for b in bs:
            d = distance(f,b)
            p = dmax - d
            if tab == []:
                heapq.heappush(tab, (p, [b,d]))
            elif tab[-1][1] > d:
                if len(tab) == n_links :
                    heapq.heappop(tab)
                heapq.heappush(tab, (p, [b,d]))
        bio_bio.append([f, tab])
    
    return fermes_bio, bio_bio


# Changement de plan

* On considère 1 b = nbt tracteurs de stockage, 1 ferme = nft tracteurs d'envoi.
* On trace uniquement les distances entre fermes et bs.
* On les trie par ordre croissant en retenant la ferme initiale et le b final.
* On envoie les tracteurs dans cet ordre, et on actualise la ferme/le b jusqu'à les faire disparaître.
* On arrête dès qu'il n'y a plus de ferme/de b (on indique la raison).

Intérêts : 
* Complexité temporelle dominée par le nombre d'arêtes
* Favorise le rapprochement spatial aux fermes (moins coûteux)
* Les cas avec les distances les moins optimales se rapportent à des positionnements de bs peu intéressants (isolement d'une ferme)


## Ebauche de l'algorithme

### Données

Une ferme :
* final int ID : pour identifier la ferme
* int[] Vf : ensemble de pointeurs vers les arrêtes où la ferme est un des sommets
* int Nft >= 0 : nombre de tracteurs encore disponibles, diminue de 1 à chaque arrête de Vf prise en compte

Un b :
* final int ID : pour identifier le b
* int[] Vb : ensemble de pointeurs vers les arrêtes où le b est un des sommets
* int Nbt >= 0 : nombre de tracteurs encore disponibles, diminue de 1 à chaque arrête de Vb prise en compte
* double D = 0 : distance entre la ferme qui sera choisie et le b 
* final int population : population concernée par le b
* final int intersections : nombre de bs "proches" 

Les arrêtes :
* final int ID : pour identifier l'arrête
* int d : distance entre les deux sommets
* ferme f : ferme reliée
* b b : b relié

### Procédure

Préalablement, trier les arêtes par ordre croissant dans une structure de données V

Tant qu'il existe une arrête dans V :
1. Récupérer l'arête de plus petit coût
2. Réduire Nft et Nbt de 1
3. Si l'un des deux tombe à 0, supprimer l'ensemble des arrêtes de V contenant l'élément concerné 

### Structure

Table de hachage pour les fermes, les bs et le arrêtes.

Création d'un tas V pour trier les arrêtes selon leur distances croissantes.


## Classes

In [4]:
class Hashtable:
    def __init__(self, size=100):
        self.size = size
        self.table = [[] for _ in range(size)]  # Initialisation de la table de hachage avec des listes vides

    def _hash(self, key):
        return hash(key) % self.size  # Fonction de hachage pour obtenir l'indice

    def add(self, key, value):
        index = self._hash(key)
        for pair in self.table[index]:
            if pair[0] == key:
                pair[1] = value  # Si la clé existe déjà, mettre à jour sa valeur
                return
        self.table[index].append([key, value])  # Ajouter la paire (clé, valeur)

    def get(self, key):
        index = self._hash(key)
        for pair in self.table[index]:
            if pair[0] == key:
                return pair[1]  # Retourner la valeur correspondant à la clé
        return None  # La clé n'existe pas dans la table de hachage

    def remove(self, key):
        index = self._hash(key)
        for i, pair in enumerate(self.table[index]):
            if pair[0] == key:
                del self.table[index][i]  # Supprimer la paire (clé, valeur)
                return
        print("La clé n'existe pas dans la table de hachage.")

In [14]:
class edge:
    id = 0
    
    def __init__(self, f_id, b_id, d:float):
        self.id = edge.id
        edge.id +=1
        self.f_id = f_id
        self.b_id = b_id
        self.d = d



class farm:
    
    def __init__(self, Nft:int, x):
        if Nft <= 0: raise "Nft <= 0" 
        self.Vf = Hashtable()
        self.Nft = Nft
        self.x = x
        
    def use(self, e:edge):
        self.Nft -= 1
        self.Vf.remove(e.id)
        
    def addVertice(self, e:edge, d:float):
        self.Vf.add(e.id, d)
        
        
        
class b:
    
    def __init__(self, Nbt:int, x, population:int, interctions : int):
        if Nbt < 0: raise "Nbt < 0" 
        self.Vb = Hashtable()
        self.Nbt = Nbt
        self.d = 0
        self.x = x
        self.population = population
        self.intersections = interctions
        
    def use(self, e:edge):
        self.Nbt -= 1
        self.Vb.remove(e.id)
        self.d = e.d
        
    def addEdge(self, e:edge, d:float):
        self.Vb.add(e.id, d)

In [11]:
class PriorityQueue:
    def __init__(self):
        self.heap = []  # Tas
        self.e_to_index = {}    # Conserve la position des arêtes (identifiées par leurs id) dans le tas
    
    def isEmpty(self):
        return len(self.heap) == 0
    
    def push(self, e:edge):
        # Ajouter un élément dans la liste
        self.heap.append(e)
        index = len(self.heap) - 1
        self.e_to_index[e.id] = index
        self._siftup(index)
    
    def pop(self):
        # Retirer le dernier élément de la liste
        if self.isEmpty() : return
        
        e = self.heap.pop()
        del self.e_to_index[e.id]
        
        return e
    
    def remove(self, e_id:edge):
        # Retirer un élément quelconque de la liste
        if e_id not in self.e_to_index:
            return  # L'élément n'existe pas dans la file de priorité
        
        # Récupération de l'élément
        index = self.e_to_index[e_id]
        
        # Echange entre le dernier élément et celui voulu
        last_e = self.heap[-1]
        self.heap[index] = last_e
        self.e_to_index[last_e.id] = index
        
        # Suppression de l'élément
        self.heap.pop()
        del self.e_to_index[e_id]
        
        if index < len(self.heap):  # Si l'élément supprimé n'était pas le dernier
            self._siftup(index)
            self._siftdown(index)
    
    def _siftup(self, index):
        # Fait remonter l'élément à sa juste position, utile pour l'insertion
        while index > 0:
            parent_index = (index - 1) // 2
            if self.heap[index].d > self.heap[parent_index].d:
                self._swap(index, parent_index)
                index = parent_index
            else:
                break
            
            
    def _swap(self, i, j):
        # Echange deux éléments du tas et modifie correctement les indices
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        self.e_to_index[self.heap[i].id] = i
        self.e_to_index[self.heap[j].id] = j
        
           
    def _siftdown(self, index):
        # Fait descendre l'élément à sa juste position, utile pour la suppression
        while True:
            left_child_index = 2 * index + 1
            right_child_index = 2 * index + 2
            
            largest = index
            
            if (left_child_index < len(self.heap) and
                self.heap[left_child_index].d > self.heap[largest].d):
                largest = left_child_index
            
            if (right_child_index < len(self.heap) and
                self.heap[right_child_index].d > self.heap[largest].d):
                largest = right_child_index
            
            if largest != index:
                self._swap(index, largest)
                index = largest
            else:
                break

## Approximation des meilleures performances

In [12]:
def distance(u,v):
    return np.sqrt((u[0]-v[0])**2 + (u[1]-v[1])**2)

In [13]:
B = []
F = []

In [15]:
# Création du tas V

E = PriorityQueue()
i_f = 0
i_b = 0

for f in F:
    i_b = 0
    for b in B:
        d = distance(f.x, b.x)
        e = edge(i_f,i_b,d)
        E.push(e)
        f.addEdge(e, d)
        b.addEdge(e, d)
        
        i_b += 1
    i_f += 1

In [17]:
# Calcul des trajets

Nf = sum([F[i].Nft for i in range(len(F))])
Nb = sum([B[i].Nbt for i in range(len(B))])


#Collecte pour potentiel affichage graphique

Efinal = []

while Nb != 0 and Nf != 0 and not E.isEmpty() :
    e = E.pop()
    Efinal.append(e)
    
    # On retire v des arêtes liées au site et on décrémente le nombre de tracteurs disponibles (localement et globalement)
    F[e.i_f].use(e)
    B[e.i_b].use(e)
    Nf-=1
    Nb-=1
    
    # S'il n'y a plus de tracteurs pour la ferme, on supprime toutes les arêtes de V le comportant
    if F[e.i_f].Nft == 0:
        for e_id in F[e.i_f].Vf:
            E.remove(e_id)
    
    # S'il n'y a plus de tracteurs pour le b, on supprime toutes les arêtes de V le comportant
    if B[e.i_b].Nbt == 0:
        for e_id in B[e.i_b].Vb:
            E.remove(e_id)   
    
    

## Evaluateur

In [19]:
def population_bonus(b):
    return b.population

def intersections_penalty(b):
    return b.intersections

def length_penalty(b):
    return b.d

def penalty(b):
    return intersections_penalty(b) + length_penalty(b) - population_bonus(b)

def global_penalty(B):
    res = 0
    for b in B:
        res += penalty(b)
    return res

## Algorithme d'essaim

In [ ]:
def new_population(n):  # Utiliser le code de Clément
    return np.zeros(n)

def evaluate(population): # A transformer en une variable de classe
    return np.vectorize(penalty)(population)

def update_position(b, global_best_x, global_best_y, inertia=0.5, cognitive=1, social=1):
    # Mise à jour de la position en fonction du meilleur résultat local et global
    dx = random.uniform(0, 1) * cognitive * (b.best_x - b.x) + random.uniform(0, 1) * social * (global_best_x - b.x)
    dy = random.uniform(0, 1) * cognitive * (b.best_y - b.y) + random.uniform(0, 1) * social * (global_best_y - b.y)
    b.x += inertia * dx
    b.y += inertia * dy

def algorithme_essaim(iterations, n):
    population = new_population(n)
    global_best_score = float('inf') 
    global_best_x = 0
    global_best_y = 0
    
    for i in range(iterations):
        evaluate(population)
        
        # Trouver le meilleur score global et sa position
        for b in population:
            if b.best_score < global_best_score:
                global_best_score = b.best_score
                global_best_x = b.best_x
                global_best_y = b.best_y
        
        # Mettre à jour les positions des bâtiments
        for b in population:
            update_position(b, global_best_x, global_best_y)
    
    return population

# Algorithme de flux maximal (plus besoin de faire ça, Cf la méthode du dessus)

On considère 4 parties principales se succédant : 
* L'entrée du flux
* Les fermes, toutes reliées à l'entrée avec comme flux maximal leur capacité de production journalière
* Les bs, reliés avec toutes les fermes *assez proches* avec une capacité infinie. Ils peuvent aussi être reliés entre eux s'ils sont assez proches.
* La sortie, reliée à tous les biodidgesteurs avec un flux maximal équivalent à leur capacité. 

Utilisation 

```python
import networkx as nx

# Création d'un graphe dirigé
G = nx.DiGraph()

# Ajout de nœuds avec leurs capacités
G.add_node("source", demand=-5)  # La source a une offre négative
G.add_node("destination", demand=5)  # La destination a une demande positive

# Ajout d'arêtes avec leurs capacités
G.add_edge("source", "A", capacity=3)
G.add_edge("source", "B", capacity=2)
G.add_edge("A", "C", capacity=2)
G.add_edge("B", "C", capacity=1)
G.add_edge("B", "D", capacity=1)
G.add_edge("C", "destination", capacity=3)
G.add_edge("D", "destination", capacity=2)

# Résolution du problème de flux maximal
flow_value, flow_dict = nx.maximum_flow(G, "source", "destination")

# Affichage des résultats
print("Flux maximal:", flow_value)
print("Flux sur chaque arête:", flow_dict)
```

# Fonction d'utilité

Grâce à l'algorithme de flux maximal, on peut évaluer les distances à parcourir et le nombre de tracteurs nécessaires pour chaque trajet.

La fonction d'utilité doit alors prendre en compte :
- la distance parcourue (carburant dépensé)
- la trop forte concentration de bs : si les bs se superposent, c'est pénalisé
- la taille de population ayant accès au b

Remarque : les bs isolés seront aussi pénalisés car ils ne seront pas reliés aux autres à cause des choix précédents.

# Optimisation par essaim

### Optimisation par Essaim (Particle Swarm Optimization - PSO) :

L'optimisation par essaim est une technique basée sur le comportement collectif des animaux en essaim, comme les oiseaux ou les poissons. Chaque individu, appelé particule, explore l'espace de recherche de solutions de manière coopérative.

**Principes de base :**
1. **Initialisation :** Placez un certain nombre de particules dans l'espace de recherche avec des positions et des vitesses initiales aléatoires.
2. **Évaluation :** Évaluez la performance (fitness) de chaque particule selon la fonction objective du problème.
3. **Mise à jour des positions et vitesses :** À chaque itération, mettez à jour les positions et les vitesses des particules en fonction de leur expérience personnelle et de l'expérience du voisinage.
4. **Mise à jour de la meilleure solution :** Mettez à jour la meilleure solution trouvée par chaque particule et par l'ensemble du groupe.
5. **Critère d'arrêt :** Continuez le processus jusqu'à ce qu'un critère d'arrêt soit satisfait.

Chaque particule ajuste sa trajectoire en fonction des meilleures solutions qu'elle a rencontrées et de l'influence de ses voisins. Cela permet une exploration efficace de l'espace de recherche pour trouver des solutions optimales.

**Remarque :** Il existe de nombreuses variantes et paramètres ajustables pour ces deux algorithmes, et leur efficacité dépend souvent de la nature spécifique du problème auquel ils sont appliqués. Les choix appropriés de paramètres et d'ajustements sont souvent un aspect important pour obtenir de bons résultats.